In [25]:
# Abre o PJE e solicita o login...
import os
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from webdriver_manager.firefox import GeckoDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = {
    "trf5": {
        "login": "https://pje.trf5.jus.br/pje/login.seam",
        "consulta": "https://pje.trf5.jus.br/pje/Processo/ConsultaProcesso/listView.seam",
        "consulta_terceiros": "https://pje.trf5.jus.br/pje/Processo/ConsultaProcessoTerceiros/listView.seam"
    },
    'jfal': {
        'login': 'https://pje.jfal.jus.br/pje/login.seam',
        'consulta': 'https://pje.jfal.jus.br/pje/Processo/ConsultaProcesso/listView.seam',
        'consulta_terceiros': 'https://pje.jfal.jus.br/pje/Processo/ConsultaProcessoTerceiros/listView.seam',
    }
}

# Configurações do Firefox para download automático
download_dir = os.path.abspath("downloads") # Define pasta de download no diretório atual

options = FirefoxOptions()
options.set_preference("browser.download.folderList", 2) # 0: Desktop, 1: Downloads padrão, 2: Pasta personalizada
options.set_preference("browser.download.dir", download_dir)
options.set_preference("browser.download.manager.showWhenStarting", False) # Desativa a janela de progresso de download
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/octet-stream") # Tipos de arquivo para salvar sem perguntar (MIME types)
options.set_preference("pdfjs.disabled", True) # Desabilita o visualizador de PDF interno do Firefox

driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()), options=options)  #Inicializa o driver com as opções
driver.get(url['jfal']['login'])

In [80]:
# Functions

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

def extrair_resultados_processos():
    """
    Extrai os dados da tabela de processos do PJE.
    
    Returns:
        list: Lista de dicionários contendo os dados dos processos.
    """
    try:
        # Esperar a tabela carregar
        wait = WebDriverWait(driver, 10)
        tabela = wait.until(EC.presence_of_element_located((By.ID, "consultaProcessoTerceirosListPanel_body")))

        # Extrair linhas de dados
        processos = []
        linhas = tabela.find_elements(By.CSS_SELECTOR, "tr .rich-table-row")

        for linha in linhas:
            celulas = linha.find_elements(By.CSS_SELECTOR, "td .rich-table-cell")
            if len(celulas) >= 6:  # Verifica se há colunas suficientes
                processo = {
                    "numero": celulas[1].text if len(celulas) > 1 else "",
                    "orgao_julgador": celulas[2].text if len(celulas) > 2 else "",
                    "distribuicao": celulas[3].text if len(celulas) > 3 else "",
                    "classe": celulas[4].text if len(celulas) > 4 else "",
                    "polo_ativo": celulas[5].text if len(celulas) > 5 else "",
                    "polo_passivo": celulas[6].text if len(celulas) > 6 else "",
                }
                processos.append(processo)
            
        return processos
        
    except Exception as e:
        print(f'Erro ao extrair dados da tabela: {str(e)}')
        return []

def wait_pje_loader(timeout=10, message_success='', message_failure=''):
    """
    Aguarda o loader do PJe (status.start) desaparecer, indicando que a página carregou.
    """
    try:
        # Define o seletor do elemento de loading (o círculo girando)
        # O PJe usa JSF, então os IDs têm ':' e '.', por isso o CSS Selector escapado
        loader_locator = (By.CSS_SELECTOR, r'#_viewRoot\:status\.start')

        # Espera até que o elemento de 'start' (loading) fique invisível
        WebDriverWait(driver, timeout).until(EC.invisibility_of_element_located(loader_locator))

        time.sleep(1)

        if message_success:
            print(message_success)
        return True

    except TimeoutException:
        print(f'Tempo excedido: {timeout} segundos')
        if message_failure:
            print(message_failure)
        return False

import csv
import os

import os

def obter_processos(arquivo_todo, arquivo_done):
    """
    Retorna uma lista de strings com os itens que estão no todo mas não no done.
    Pega apenas a primeira coluna (ID) do arquivo_done para verificação.
    """
    concluidos = set()

    # 1. Carrega o que já foi feito (pegando apenas a 1ª coluna)
    if os.path.exists(arquivo_done):
        with open(arquivo_done, 'r', encoding='utf-8') as f:
            for linha in f:
                processo = linha.split(',')[0].strip()
                concluidos.add(processo)

    itens_para_processar = []

    # 2. Filtra o que falta fazer
    if os.path.exists(arquivo_todo):
        with open(arquivo_todo, 'r', encoding='utf-8') as f:
            for linha in f:
                # Assume que o arquivo_todo tem apenas o ID por linha
                # Se tiver colunas também, use .split(',')[0]
                item = linha.strip()
                if item and item not in concluidos:
                    itens_para_processar.append(item)
    
    return itens_para_processar

def consultar_processo_numero(numero_processo, terceiro=False):
    try:

        if terceiro:
            print('Buscando em processos de terceiros')
            driver.get(url['jfal']['consulta_terceiros'])
        else:
            print('Buscando em processos')
            driver.get(url['jfal']['consulta'])

        wait_pje_loader()

        driver.execute_script(f"""document.querySelector('input[id*="ProcessoDecoration"]').value = '{numero_processo}'""")
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@value='Pesquisar']"))).click()

        wait_pje_loader()
                    
    except Exception as e:
        print(f'Erro durante a consulta de processos: {str(e)}')
        return []

import time

def download_autos():
    driver.switch_to.window(driver.window_handles[-1])
    driver.maximize_window()

    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@value='Download de documentos em PDF']"))).click()
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@value='Selecionar']"))).click()
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@value='Gerar PDF']"))).click()

    print('Aguardando início do download...')
    time.sleep(10)

def fechar_janelas():
    if len(driver.window_handles) > 0:
        original_window = driver.window_handles[0]
        for window in driver.window_handles:
            if window != original_window:
                driver.switch_to.window(window)
                driver.close()
        
        driver.switch_to.window(original_window)

In [83]:
all = 'processos.csv'
done = 'processos_baixados.csv'
todo = obter_processos(all, done)

print(len(todo))

for numero in todo:
    try:
        print(numero)
        fechar_janelas()
        
        # Tenta em consulta normal
        consultar_processo_numero(numero)
        time.sleep(5)

        if len(driver.find_elements(By.CSS_SELECTOR, r'#consultaProcessoList\:tb .rich-table-row')) > 0:
            driver.find_element(By.CSS_SELECTOR, r'#consultaProcessoList\:tb .rich-table-firstrow img').click()
            
            time.sleep(5)
            download_autos()
            with open('processos_baixados.csv', 'a', encoding='utf-8') as file:
                file.write(f"{numero},'proprio'\n")
                file.flush()
            continue

        # Tenta em consulta de processo de terceiros
        consultar_processo_numero(numero, True)
        time.sleep(5)

        if len(driver.find_elements(By.CSS_SELECTOR, r'#consultaProcessoTerceirosList\:tb .rich-table-row')) > 0:
            driver.find_element(By.CSS_SELECTOR, r'#consultaProcessoTerceirosList\:tb .rich-table-firstrow img').click()
            time.sleep(2)

            driver.find_element(By.ID, 'modal:motivacaoDecoration:motivacao').send_keys('.')
            driver.find_element(By.ID, 'modal:btnGravar').click()
            
            time.sleep(5)
            download_autos()
            with open('processos_baixados.csv', 'a', encoding='utf-8') as file:
                file.write(f"{numero},'terceiros'\n")
                file.flush()
            continue

        print('Nenhum processo encontrado')


    except Exception as e:
        print(f'Erro durante a iteração: {str(e)}')
        continue

659
0815872-67.2021.4.05.8000
Buscando em processos
Buscando em processos de terceiros
Aguardando início do download...
0815869-15.2021.4.05.8000
Buscando em processos
Buscando em processos de terceiros
Aguardando início do download...
0815867-45.2021.4.05.8000
Buscando em processos
Buscando em processos de terceiros
Erro durante a iteração: Message: The element with the reference 4e18cac8-ffb4-4a1b-a45d-375ce81de7a3 is stale; either its node document is not the active document, or it is no longer connected to the DOM; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#staleelementreferenceexception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:202:5
StaleElementReferenceError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:831:5
getKnownElement@chrome://remote/content/marionette/json.sys.mjs:412:11
deseriali

KeyboardInterrupt: 

In [82]:
# Renomeia os arquivos da pasta

import os
import re
from pypdf import PdfReader

# Garante que a pasta existe (usando a variável que você já definiu antes)
pasta = download_dir 

def limpar_nome_arquivo(nome):
    """Remove caracteres inválidos para nomes de arquivos no Windows"""
    return re.sub(r'[<>:"/\\|?*]', '_', nome)

# Regex para encontrar o número do processo
padrao_processo = re.compile(r'\d{7}-\d{2}\.\d{4}\.\d\.\d{2}\.\d{4}')

# Itera sobre os arquivos da pasta
for nome_arquivo in os.listdir(pasta):
    if nome_arquivo.lower().endswith(".pdf"):
        caminho_antigo = os.path.join(pasta, nome_arquivo)
        titulo_pdf = None

        try:
            # 1. Abre o arquivo e extrai o título
            with open(caminho_antigo, "rb") as f:
                reader = PdfReader(f)
                if reader.metadata and reader.metadata.title:
                    titulo_pdf = reader.metadata.title

            # 2. Renomeia (agora que o arquivo já está fechado)
            if titulo_pdf:
                # Tenta encontrar o número do processo no título usando o regex
                match = padrao_processo.search(titulo_pdf)
                
                if match:
                    # Se encontrou o padrão, usa ele como nome do arquivo
                    novo_nome = match.group() + ".pdf"
                else:
                    # Se não encontrou, usa o título limpo original
                    novo_nome = limpar_nome_arquivo(titulo_pdf) + ".pdf"

                caminho_novo = os.path.join(pasta, novo_nome)

                # Verifica se já existe um arquivo com esse nome para não sobrescrever
                # E também verifica se o nome já é o correto (caminho antigo != novo)
                if caminho_antigo != caminho_novo:
                    if not os.path.exists(caminho_novo):
                        os.rename(caminho_antigo, caminho_novo)
                        print(f"Renomeado: '{nome_arquivo}' -> '{novo_nome}'")
                    else:
                        print(f"Pulei: '{novo_nome}' já existe na pasta.")
            else:
                print(f"Arquivo '{nome_arquivo}' não possui metadados de Título.")

        except Exception as e:
            print(f"Erro ao processar {nome_arquivo}: {e}")

Renomeado: '+cSEPhjyvN-p+VroauAhSdrD.node03_00027512403994226498.pdf' -> '0815401-51.2021.4.05.8000.pdf'
Renomeado: '+cSEPhjyvN-p+VroauAhSdrD.node03_0005816677759146183.pdf' -> '0813738-67.2021.4.05.8000.pdf'
Renomeado: '+cSEPhjyvN-p+VroauAhSdrD.node03_0005837544968020025.pdf' -> '0814642-87.2021.4.05.8000.pdf'
Renomeado: '+cSEPhjyvN-p+VroauAhSdrD.node03_0007584266114017857.pdf' -> '0814861-03.2021.4.05.8000.pdf'
Renomeado: '+cSEPhjyvN-p+VroauAhSdrD.node03_00076875620058760585.pdf' -> '0814645-42.2021.4.05.8000.pdf'
Renomeado: '+cSEPhjyvN-p+VroauAhSdrD.node03_0012897241459365327.pdf' -> '0814862-85.2021.4.05.8000.pdf'
Renomeado: '+cSEPhjyvN-p+VroauAhSdrD.node03_001618513637800245.pdf' -> '0815091-45.2021.4.05.8000.pdf'
Renomeado: '+cSEPhjyvN-p+VroauAhSdrD.node03_0021908726359093356.pdf' -> '0813752-51.2021.4.05.8000.pdf'
Renomeado: '+cSEPhjyvN-p+VroauAhSdrD.node03_0027843519907869707.pdf' -> '0815125-20.2021.4.05.8000.pdf'
Renomeado: '+cSEPhjyvN-p+VroauAhSdrD.node03_0029562931707368678

In [56]:
# Gera lista dos arquivos da pasta

import os
import csv

# Define a pasta onde estão os arquivos
pasta = download_dir
arquivo_saida = "arquivos_baixados.csv"

# Lista para armazenar os nomes dos arquivos
lista_arquivos = []

# Varre a pasta e pega os arquivos PDF
if os.path.exists(pasta):
    for nome_arquivo in os.listdir(pasta):
        if nome_arquivo.lower().endswith(".pdf"):
            lista_arquivos.append([nome_arquivo])
else:
    print(f"A pasta '{pasta}' não foi encontrada.")

# Escreve no CSV
if lista_arquivos:
    with open(arquivo_saida, mode='w', newline='', encoding='utf-8') as f:
        escritor = csv.writer(f)
        escritor.writerow(["Nome do Arquivo"])  # Cabeçalho
        escritor.writerows(lista_arquivos)
    
    print(f"Arquivo CSV criado com sucesso: {arquivo_saida}")
    print(f"Total de arquivos listados: {len(lista_arquivos)}")
else:
    print("Nenhum arquivo PDF encontrado na pasta.")

Arquivo CSV criado com sucesso: arquivos_baixados.csv
Total de arquivos listados: 94
